In [ ]:
import vllm, torch
from vllm import LLM, SamplingParams

MODELTORUN = "/home/ubuntu/llm/models/Qwen2-7B-Instruct"
torch.cuda.empty_cache()
llm = LLM(model=MODELTORUN,
         dtype=torch.float16,max_model_len=1000,
         seed=415,
         max_num_batched_tokens=3000)

WARNING 11-03 20:01:20 config.py:1650] Casting torch.bfloat16 to torch.float16.
INFO 11-03 20:01:20 llm_engine.py:232] Initializing an LLM engine (v0.6.1) with config: model='/home/ubuntu/llm/models/qwen2_vl_lora_sft', speculative_config=None, tokenizer='/home/ubuntu/llm/models/qwen2_vl_lora_sft', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, rope_scaling=None, rope_theta=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.float16, max_seq_len=1000, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), observability_config=ObservabilityConfig(otlp_traces_endpoint=None, collect_model_forward_time=False, collect_model_execute_time=False), seed=415, served_model_name=/home/ubu

/home/ubuntu/miniconda3/envs/llm/lib/python3.10/site-packages/xformers/ops/fmha/flash.py:211: FutureWarning: `torch.library.impl_abstract` was renamed to `torch.library.register_fake`. Please use that instead; we will remove `torch.library.impl_abstract` in a future version of PyTorch.
  @torch.library.impl_abstract("xformers_flash::flash_fwd")
/home/ubuntu/miniconda3/envs/llm/lib/python3.10/site-packages/xformers/ops/fmha/flash.py:344: FutureWarning: `torch.library.impl_abstract` was renamed to `torch.library.register_fake`. Please use that instead; we will remove `torch.library.impl_abstract` in a future version of PyTorch.
  @torch.library.impl_abstract("xformers_flash::flash_bwd")


INFO 11-03 20:01:21 model_runner.py:997] Starting to load model /home/ubuntu/llm/models/qwen2_vl_lora_sft...
INFO 11-03 20:01:21 selector.py:217] Cannot use FlashAttention-2 backend for Volta and Turing GPUs.
INFO 11-03 20:01:21 selector.py:116] Using XFormers backend.


Loading safetensors checkpoint shards:   0% Completed | 0/10 [00:00<?, ?it/s]


INFO 11-03 20:01:29 model_runner.py:1008] Loading model weights took 15.7193 GB
INFO 11-03 20:01:31 gpu_executor.py:122] # GPU blocks: 13180, # CPU blocks: 4681
INFO 11-03 20:01:34 model_runner.py:1309] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 11-03 20:01:34 model_runner.py:1313] CUDA graphs can take additional 1~3 GiB memory per GPU. If you are running out of memory, consider decreasing `gpu_memory_utilization` or enforcing eager mode. You can also reduce the `max_num_seqs` as needed to decrease memory usage.
INFO 11-03 20:01:51 model_runner.py:1428] Graph capturing finished in 16 secs.


In [2]:
from milvus_utils import *
import encoder

SAMPLE_QUESTION = "这是什么故障？故障IV曲线的开路电压Voc比正常状态偏差率小于20%，且故障IV曲线的短路电流Isc比正常状态偏差率也小于20%，。而且故障IV曲线的拐点会更圆滑，也即电流快速下降处的电压相较于正常状态更小，斜率更平缓。"
em=encoder.EmbeddingModel(model = "/home/ubuntu/llm/models/bge-m3",device='cpu')
vector,ok=em.emb_text(SAMPLE_QUESTION)

DB_NAME="/home/ubuntu/rag/lq/milvus_demo.db"
COLLECTION_NAME="RAG_demo"
milvus_client=MilvusClient(DB_NAME)
results=get_search_results(milvus_client=milvus_client,
                           collection_name=COLLECTION_NAME,
                           query_vector=vector,output_fields=['text', 'source','vector'])
contexts=[]
sources=[]
for result in results:
    for hit in result:
        contexts.append(hit["entity"]["text"])
        sources.append(hit["entity"]["source"])


In [3]:
contexts_combined = ' '.join(contexts)
contexts_combined = ' '.join(reversed(contexts))
source_combined = ' '.join(reversed(list(dict.fromkeys(sources))))


SYSTEM_PROMPT = f"""首先，检查提供的Context 是否与以下User's question相关。其次，只有当提供的Context具有很强的相关性时，才能使用Context回答问题。否则，如果Context不太相关，请在不使用Context的情况下回答问题。
清晰、简洁、相关。
Grounding sources: {source_combined}
Context: {contexts_combined}
User's question: {SAMPLE_QUESTION}
"""


prompts = [SYSTEM_PROMPT]
print(prompts)

['首先，检查提供的Context 是否与以下User\'s question相关。其次，只有当提供的Context具有很强的相关性时，才能使用Context回答问题。否则，如果Context不太相关，请在不使用Context的情况下回答问题。\n清晰、简洁、相关。\nGrounding sources: /home/ubuntu/rag/lq/iiip/pv.md\nContext: 老化故障\n\n"content": "这是老化故障与正常状态IV曲线特征的对比图，因为老化故障IV曲线的开路电压Voc与正常状态IV曲线的开路电压Voc偏差率小于20%，且老化故障IV曲线的短路电流Isc与正常状态IV曲线的短路电流Isc偏差率也小于20%，。但老化故障IV曲线的拐点会更圆滑，也即电流快速下降处的电压相较于正常状态更小，斜率更平缓。"\n\n开路故障 短路故障\n\n"content": "这是短路故障与正常状态IV曲线特征的对比图，因为短路故障IV曲线的短路电流Isc与正常状态IV曲线的短路电流Isc偏差率小于20%，但短路故障IV曲线的开路电压Voc显著小于正常状态IV曲线的开路电压Voc，偏差率大于20%。"\n\n遮挡故障 开路故障\n\n"content": "这是开路故障与正常状态IV曲线特征的对比图，因为开路故障IV曲线的开路电压Voc与正常状态IV曲线的开路电压Voc偏差率小于20%，但开路故障IV曲线的短路电流Isc显著小于正常状态IV曲线的短路电流Isc，偏差率大于20%。"\n\n短路故障\nUser\'s question: 这是什么故障？故障IV曲线的开路电压Voc比正常状态偏差率小于20%，且故障IV曲线的短路电流Isc比正常状态偏差率也小于20%，。而且故障IV曲线的拐点会更圆滑，也即电流快速下降处的电压相较于正常状态更小，斜率更平缓。\n']


In [4]:
sampling_params = SamplingParams(temperature=0.7, top_p=0.8, repetition_penalty=1.05, max_tokens=4096)


outputs = llm.generate(prompts, sampling_params)


for output in outputs:
   prompt = output.prompt
   generated_text = output.outputs[0].text
   # !r calls repr(), which prints a string inside quotes.
   print(f"Question: {SAMPLE_QUESTION!r}")
   print(f"Generated text: {generated_text!r}")

Processed prompts: 100%|██████████| 1/1 [00:02<00:00,  2.81s/it, est. speed input: 159.21 toks/s, output: 37.40 toks/s]

Question: '这是什么故障？故障IV曲线的开路电压Voc比正常状态偏差率小于20%，且故障IV曲线的短路电流Isc比正常状态偏差率也小于20%，。而且故障IV曲线的拐点会更圆滑，也即电流快速下降处的电压相较于正常状态更小，斜率更平缓。'
Generated text: '这是老化故障与正常状态IV曲线特征的对比图，因为老化故障IV曲线的开路电压Voc与正常状态IV曲线的开路电压Voc偏差率小于20%，且老化故障IV曲线的短路电流Isc与正常状态IV曲线的短路电流Isc偏差率也小于20%，。但老化故障IV曲线的拐点会更圆滑，也即电流快速下降处的电压相较于正常状态更小，斜率更平缓。'
